In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset OCE
df = pd.read_excel(r"C:\Users\sabri\OCETeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,win,THE LAKESHOW,ROCK SNIFFERS,1237,3,2,4,13,6,23.076923,3,4,0.666667
1,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,loss,ROCK SNIFFERS,THE LAKESHOW,1010,1,1,7,6,13,16.666667,4,3,0.538462
2,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,win,PENSION,ROCK SNIFFERS,959,1,0,8,4,10,25.000000,0,3,0.800000
3,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,loss,ROCK SNIFFERS,PENSION,697,0,0,3,10,4,0.000000,3,0,0.750000
4,fcf36a21-2873-47f8-bd81-bf22c24a3044,OCE [AA] GZ vs TC G1 2022-10-28.20.14,Mannfield (Night),2022-10-28,win,GROUND ZERO,TRE CANI,1463,2,2,7,9,9,22.222222,5,7,0.777778


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,win,THE LAKESHOW,ROCK SNIFFERS,1237,3,2,4,13,6,23.076923,3,4,0.666667
1,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,loss,ROCK SNIFFERS,THE LAKESHOW,1010,1,1,7,6,13,16.666667,4,3,0.538462
2,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,win,PENSION,ROCK SNIFFERS,959,1,0,8,4,10,25.000000,0,3,0.800000
3,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,loss,ROCK SNIFFERS,PENSION,697,0,0,3,10,4,0.000000,3,0,0.750000
4,fcf36a21-2873-47f8-bd81-bf22c24a3044,OCE [AA] GZ vs TC G1 2022-10-28.20.14,Mannfield (Night),2022-10-28,win,GROUND ZERO,TRE CANI,1463,2,2,7,9,9,22.222222,5,7,0.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2617,04e844f8-2aaa-41af-a68e-6c03e21d2603,OCE D HIM vs DL G3 2023-06-02.17.21,DFH Stadium,2023-06-02,loss,DUMB LUCK,HIM POLICE,728,0,0,6,6,10,0.000000,4,1,0.600000
2618,65c9dd77-bff9-4a58-a789-3d7ef0eef271,OCE AB MRG vs PWR G4 2023-06-03.15.04,Utopia Coliseum (Dusk),2023-06-03,win,MERGE ESPORTS,PWR,1508,6,4,4,6,10,100.000000,4,5,0.400000
2619,65c9dd77-bff9-4a58-a789-3d7ef0eef271,OCE AB MRG vs PWR G4 2023-06-03.15.04,Utopia Coliseum (Dusk),2023-06-03,loss,PWR,MERGE ESPORTS,628,1,1,1,10,6,10.000000,5,4,0.166667
2620,6ab4a0a2-6dc8-4d98-999c-59df75996fc4,OCE O GZ vs Tsui G2 2023-06-04.11.16,Forbidden Temple,2023-06-04,loss,GROUND ZERO,TSURANI,1002,1,1,7,7,11,14.285714,3,1,0.636364


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,win,THE LAKESHOW,ROCK SNIFFERS,1237,1,2,4,13,6,23.076923,3,4,0.666667
1,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,loss,ROCK SNIFFERS,THE LAKESHOW,1010,0,1,7,6,13,16.666667,4,3,0.538462
2,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,win,PENSION,ROCK SNIFFERS,959,1,0,8,4,10,25.000000,0,3,0.800000
3,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,loss,ROCK SNIFFERS,PENSION,697,0,0,3,10,4,0.000000,3,0,0.750000
4,fcf36a21-2873-47f8-bd81-bf22c24a3044,OCE [AA] GZ vs TC G1 2022-10-28.20.14,Mannfield (Night),2022-10-28,win,GROUND ZERO,TRE CANI,1463,0,2,7,9,9,22.222222,5,7,0.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2617,04e844f8-2aaa-41af-a68e-6c03e21d2603,OCE D HIM vs DL G3 2023-06-02.17.21,DFH Stadium,2023-06-02,loss,DUMB LUCK,HIM POLICE,728,0,0,6,6,10,0.000000,4,1,0.600000
2618,65c9dd77-bff9-4a58-a789-3d7ef0eef271,OCE AB MRG vs PWR G4 2023-06-03.15.04,Utopia Coliseum (Dusk),2023-06-03,win,MERGE ESPORTS,PWR,1508,2,4,4,6,10,100.000000,4,5,0.400000
2619,65c9dd77-bff9-4a58-a789-3d7ef0eef271,OCE AB MRG vs PWR G4 2023-06-03.15.04,Utopia Coliseum (Dusk),2023-06-03,loss,PWR,MERGE ESPORTS,628,0,1,1,10,6,10.000000,5,4,0.166667
2620,6ab4a0a2-6dc8-4d98-999c-59df75996fc4,OCE O GZ vs Tsui G2 2023-06-04.11.16,Forbidden Temple,2023-06-04,loss,GROUND ZERO,TSURANI,1002,0,1,7,7,11,14.285714,3,1,0.636364


In [5]:
#  Checando se há empates nos resultados e removendo-os
df_draw = df[df['result']=='draw']
df = df.drop(df_draw.index)

In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,THE LAKESHOW,ROCK SNIFFERS,1237,1,2,4,13,6,23.076923,3,4,0.666667,0
1,fc839dc8-2ffd-48a8-be11-25e411180b13,OCE [B] TLS vs RS G5 2022-10-28.16.33,Champions Field,2022-10-28,ROCK SNIFFERS,THE LAKESHOW,1010,0,1,7,6,13,16.666667,4,3,0.538462,1
2,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,PENSION,ROCK SNIFFERS,959,1,0,8,4,10,25.000000,0,3,0.800000,0
3,63deb665-7086-4750-96cd-f6c648b0c62b,OCE W Rock Sniffers vs Pension G5 2022-10-28.1...,Champions Field,2022-10-28,ROCK SNIFFERS,PENSION,697,0,0,3,10,4,0.000000,3,0,0.750000,1
4,fcf36a21-2873-47f8-bd81-bf22c24a3044,OCE [AA] GZ vs TC G1 2022-10-28.20.14,Mannfield (Night),2022-10-28,GROUND ZERO,TRE CANI,1463,0,2,7,9,9,22.222222,5,7,0.777778,0


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2372, 12) (250, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(250, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1779, 9), (593, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.85      0.88       312
           1       0.85      0.89      0.87       281

    accuracy                           0.87       593
   macro avg       0.87      0.87      0.87       593
weighted avg       0.87      0.87      0.87       593



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

cv = KFold(n_splits=10, shuffle=True, random_state=10)

In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=20,
                           n_estimators=300)


In [19]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 20, learning_rate = 0.05, n_estimators = 300)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       296
           1       0.90      0.85      0.87       297

    accuracy                           0.88       593
   macro avg       0.88      0.88      0.88       593
weighted avg       0.88      0.88      0.88       593
               precision    recall  f1-score   support

           0       0.87      0.92      0.89       125
           1       0.92      0.86      0.89       125

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250



In [20]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       890
           1       0.93      0.94      0.94       889

    accuracy                           0.94      1779
   macro avg       0.94      0.94      0.94      1779
weighted avg       0.94      0.94      0.94      1779



In [21]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.0214616312000733, 0.11147710205502782, 0.4324357338356117, 0.05995163067133146, 0.03677422540572513, 0.08056440668193887, 0.003727855165088874, 0.0044456204638499116, 0.2491617945213529]
